In [ ]:
import tensorflow_datasets as tfds

In [ ]:
imdb, info = tfds.load('imdb_reviews',as_supervised=True, with_info=True)

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

for s,l in train_data:
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

In [ ]:
train_sentences[0]

'b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."'

In [ ]:
train_labels[0]

0

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
vocab_size = 10000
embedding_dim = 64
max_length = 140

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [ ]:
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating='post',padding ='post')

In [ ]:
padded_train_sequences

array([[  59,   12,   14, ...,    0,    0,    0],
       [ 256,   28,   78, ...,    0,    0,    0],
       [   1, 6175,    2, ...,    0,    0,    0],
       ...,
       [7629,   37,   11, ..., 2743,   21, 1563],
       [2677,   11,  216, ..., 1048,   37,   31],
       [3875,    5,   31, ...,    1, 2238,   42]], dtype=int32)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating='post',padding ='post')

In [ ]:
print(padded_train_sequences.shape)
print(padded_test_sequences.shape)

(25000, 140)
(25000, 140)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense



In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(vocab_size,embedding_dim,input_length=max_length))

In [ ]:
model.add(Flatten())

In [ ]:
model.add(Dense(10,activation='relu'))

In [ ]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics =['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 140, 64)           640000    
                                                                 
 flatten (Flatten)           (None, 8960)              0         
                                                                 
 dense (Dense)               (None, 10)                89610     
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 729,621
Trainable params: 729,621
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded_train_sequences, train_labels, epochs=10, validation_data=(padded_test_sequences,test_labels))

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.4497 - acc: 0.7741 - val_loss: 0.3752 - val_acc: 0.8330
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.1342 - acc: 0.9563 - val_loss: 0.4684 - val_acc: 0.8151
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0204 - acc: 0.9966 - val_loss: 0.5511 - val_acc: 0.8198
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0029 - acc: 0.9998 - val_loss: 0.6073 - val_acc: 0.8200
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 7.9310e-04 - acc: 1.0000 - val_loss: 0.6522 - val_acc: 0.8226
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 3.5084e-04 - acc: 1.0000 - val_loss: 0.6870 - val_acc: 0.8240
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 1.9183e-04 - acc: 1.0000 - val_loss: 0.7168 - val_acc: 0.8250
Epoch 8/10
782/782 [==============================] - 6s 8ms/ste

In [ ]:
def model_predict(text):
  test_seq = tokenizer.texts_to_sequences(text)
  padded_test_seq = pad_sequences(test_seq,maxlen=max_length,padding='post',truncating='post' )

  pred = model.predict(padded_test_seq)
  print('Positive') if pred > 0.5 else print('Negative')

In [ ]:
test_sen =["This is my second attempt to review this title. I was able to see a screening of this in Europe...waste of time. Predictable and laughable at points, it desperately tries to keep pace with other Marvel offerings but fails miserably. "]
model_predict(test_sen)

1/1 [==============================] - 0s 14ms/step
Negative


In [ ]:
e = model.layers[0]